In [ ]:
import numpy as np
from qore import Mine, ASP, QAOA, VQE
from qore.utils import measure_operator, get_bitstring_probabilities

In [ ]:
penalty = 10.0
qmine = Mine(np.array([[-2., 3., 1.], [float('inf'), 5., float('inf')]]))

# penalty = 20.0
# qmine = Mine(np.array([[-2., 3., 1., -1.], [float('inf'), 5., 3., float('inf')]]))

# penalty = 5.0
# qmine = Mine(np.array([[-2.0, 3.0, -1.0, -2.0, -1.0], [float('inf'), 5.0, 3.0, 10.0, float('inf')], [float('inf'), float('inf'), 4.0, float('inf'), float('inf')]]))

qmine.plot_mine()

In [ ]:
qmine.graph

In [ ]:
qmine.gen_Hamiltonian(penalty=penalty).to_opflow()

In [ ]:
print(qmine.gen_Hamiltonian(penalty=penalty).to_opflow())

In [ ]:
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit import Aer

aqua_globals.random_seed = 1953
backend = Aer.get_backend('statevector_simulator')
instance = QuantumInstance(backend=backend)

def analysis(circuit):
    print("Average Profit:                          ", measure_operator(qmine.Hp, circuit, instance))
    print("Average Number of Smoothness Violations: ", measure_operator(qmine.Hs, circuit, instance) / penalty)
    x = get_bitstring_probabilities(circuit, instance, shots=1024)
    bitstr, prob = max(x.items(), key=lambda item: item[1])
    print(f"The most probable configuration and the corresponding probability: {bitstr, prob}")
    qmine.plot_mine_state(bitstr)
    print("Current profit:               ", qmine.get_profit(bitstr))
    print("Current number of violations: ", qmine.get_violation(bitstr))
    print(x)
    print("------------------------------------------------------------")


In [ ]:
evol_time = 20
nsteps = 50


circuit = ASP(qmine.gen_Hamiltonian(penalty=penalty), 
    evol_time=evol_time, 
    nsteps=nsteps, 
    callback=analysis, 
    quantum_instance=instance).construct_circuit()

analysis(circuit)
print(circuit.draw())
print(circuit.decompose().draw())

In [ ]:
from qiskit.aqua.components.optimizers import COBYLA

p = 5
qaoa = QAOA(qmine.gen_Hamiltonian(penalty=penalty), optimizer=COBYLA(), quantum_instance=instance, p=p)
qaoa.run()
circuit = qaoa.get_optimal_circuit()

analysis(circuit)
print(circuit.draw())
print(circuit.decompose().draw())